In [42]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
from tensorflow.keras import optimizers, callbacks
import numpy as np
import pandas as pd
import pathlib
import os
from os import getcwd
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import multiprocessing
import time
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## https://www.kaggle.com/annettecatherinepaul/online-shoppers-behavior-prediction

__Learning how and when shoppers will research and purchase goods online is important to businesses as they can use customer behavior insights to target advertising, marketing, and deals to potential customers to further increase their sales and revenue__

In [43]:
df = pd.read_csv(r'C:\data\Data_Class\online_shoppers_intention.csv', index_col=None)
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
Administrative             12330 non-null int64
Administrative_Duration    12330 non-null float64
Informational              12330 non-null int64
Informational_Duration     12330 non-null float64
ProductRelated             12330 non-null int64
ProductRelated_Duration    12330 non-null float64
BounceRates                12330 non-null float64
ExitRates                  12330 non-null float64
PageValues                 12330 non-null float64
SpecialDay                 12330 non-null float64
Month                      12330 non-null object
OperatingSystems           12330 non-null int64
Browser                    12330 non-null int64
Region                     12330 non-null int64
TrafficType                12330 non-null int64
VisitorType                12330 non-null object
Weekend                    12330 non-null bool
Revenue                    12330 non-null bool
dtypes: bool(

In [64]:

df['Weekend']=df['Weekend'].astype(int)
df['Revenue']=df['Revenue'].astype(int)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
Administrative             12330 non-null int64
Administrative_Duration    12330 non-null float64
Informational              12330 non-null int64
Informational_Duration     12330 non-null float64
ProductRelated             12330 non-null int64
ProductRelated_Duration    12330 non-null float64
BounceRates                12330 non-null float64
ExitRates                  12330 non-null float64
PageValues                 12330 non-null float64
SpecialDay                 12330 non-null float64
Month                      12330 non-null object
OperatingSystems           12330 non-null int64
Browser                    12330 non-null int64
Region                     12330 non-null int64
TrafficType                12330 non-null int64
VisitorType                12330 non-null object
Weekend                    12330 non-null int32
Revenue                    12330 non-null bool
dtypes: bool

In [47]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7891 train examples
1973 validation examples
2466 test examples


## Create an input pipeline using tf.data
__Next, wrap the dataframes with tf.data.__

__This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model.__ 


In [65]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, label,shuffle=True, batch_size=32, ):
  df = dataframe.copy()
  labels = df.pop(label)
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df)-100)
  ds = ds.batch(batch_size)
  return ds

In [66]:
batch_size = 4 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train,'Revenue', batch_size=batch_size)
val_ds = df_to_dataset(val,'Revenue', shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test,'Revenue', shuffle=False, batch_size=batch_size)

## After creating the the input pipeline, check to see the format of the data it returns with  a small batch size.

In [67]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of Amount:', feature_batch['VisitorType'])
  print('A batch of Class:', label_batch )

Every feature: ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']
A batch of Amount: tf.Tensor(
[b'Returning_Visitor' b'Returning_Visitor' b'New_Visitor'
 b'Returning_Visitor'], shape=(4,), dtype=string)
A batch of Class: tf.Tensor([False False False  True], shape=(4,), dtype=bool)


In [68]:
df.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')

In [69]:
feature_columns = []

# numeric cols
for header in ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']:
  feature_columns.append(feature_column.numeric_column(header))

In [70]:
# indicator_columns
indicator_column_names = [ 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend' ]
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, df[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [71]:
feature_columns

[NumericColumn(key='Administrative', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Administrative_Duration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Informational', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Informational_Duration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ProductRelated', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ProductRelated_Duration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='BounceRates', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ExitRates', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PageValues', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(

## Create a feature layer, we have defined our feature columns, we will use a DenseFeatures layer to input them to our Keras model.

In [72]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [73]:
batch_size = 32
train_ds = df_to_dataset(train,'Revenue', batch_size=batch_size)
val_ds = df_to_dataset(val, 'Revenue', shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, 'Revenue',shuffle=False, batch_size=batch_size)

In [74]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dropout(.3),
  layers.Dense(64, activation='relu'),
  layers.Dropout(.3),
  layers.Dense(1)
])
optimizer = optimizers.Adam(lr=0.0005, beta_1=0.7, beta_2=0.999, amsgrad=True)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=50)

Epoch 1/50
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Consider rewriting this model with the Functional API.
240/247 [============================>.] - ETA: 0s - loss: 5.1018 - accuracy: 0.7457WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Administrative': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'Administrative_Duration': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'Informational

Epoch 22/50
247/247 [==============================] - 1s 6ms/step - loss: 0.3618 - accuracy: 0.8550 - val_loss: 0.3446 - val_accuracy: 0.8424
Epoch 23/50
247/247 [==============================] - 1s 6ms/step - loss: 0.3534 - accuracy: 0.8552 - val_loss: 0.3835 - val_accuracy: 0.8398
Epoch 24/50
247/247 [==============================] - 1s 6ms/step - loss: 0.3580 - accuracy: 0.8527 - val_loss: 0.3362 - val_accuracy: 0.8419
Epoch 25/50
247/247 [==============================] - 1s 6ms/step - loss: 0.3535 - accuracy: 0.8560 - val_loss: 0.3306 - val_accuracy: 0.8444
Epoch 26/50
247/247 [==============================] - 1s 6ms/step - loss: 0.3485 - accuracy: 0.8552 - val_loss: 0.3709 - val_accuracy: 0.8429
Epoch 27/50
247/247 [==============================] - 1s 6ms/step - loss: 0.3380 - accuracy: 0.8560 - val_loss: 0.3243 - val_accuracy: 0.8454
Epoch 28/50
247/247 [==============================] - 1s 5ms/step - loss: 0.3332 - accuracy: 0.8569 - val_loss: 0.3601 - val_accuracy: 0.8459

## https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/feature_columns.ipynb#scrollTo=_YJPPb3xTPeZ